In [3]:
!pip install scipy
!pip install qiskit_aer
!pip install qiskit

In [35]:
import numpy as np
import math
import pandas as pd
from random import randint
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from qiskit.circuit import ParameterVector
from scipy.optimize import minimize
from qiskit_aer.aerprovider import AerSimulator
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import RealAmplitudes
from qiskit.circuit.library import *
from qiskit.circuit import ClassicalRegister, QuantumRegister, Parameter, ParameterVector
from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import *
from Utilities import *
import heapq

In [36]:
"""
Function to naively add fully parametrized, maximally entangled layer layers times
Calculates gradients for each layer, removes smallest magnitude RY gates with given pruning rate
"""
def NaiveBuilder(params:list, ansatz:QuantumCircuit, layers:int,
                 circuit:QuantumCircuit, hamiltonian:SparsePauliOp, estimator:Estimator, pruning_rates):
    n = circuit.num_qubits
    results = []
    
    for rate in pruning_rates:
        temp_ansatz = ansatz.copy()
        temp_params = params.copy()
        new_params = ParameterVector(f'new_{rate}', layers * n)
        
        for l in range(layers):
            naive_layer = QuantumCircuit(n)
            for i in range (n):
                naive_layer.ry(new_params[l * n + i], i)
                temp_params.append(1)
            for i in range(1, n):
                naive_layer.cx(0, i)
                
            temp_ansatz = temp_ansatz.compose(naive_layer)
            accumulator = [(randint(0, 10), i) for i in range(len(temp_params) - n, len(temp_params))]
            heapq.heapify(accumulator)
            bound = math.floor(rate * n)
            remove = [heapq.heappop(accumulator)[1] % n for _ in range(bound)]
            
            for i, idx in enumerate(sorted(remove)):
                del naive_layer.data[idx - i]
                del temp_params[-1]
                
            temp_ansatz = temp_ansatz.compose(naive_layer)
            
        final_circuit = circuit.compose(temp_ansatz)
        results.append((rate, final_circuit.depth(), len(temp_params)))
        
    return results

In [39]:
def visualize_results(results):
    rates, depths, params = zip(*results)
    plt.figure(figsize=(10, 5))
    plt.subplot(1,2,1)
    plt.plot(rates, depths, marker='o', label='Circuit Depth')
    plt.xlabel('Pruning Rate')
    plt.ylabel('Circuit Depth')
    plt.legend()
    
    plt.subplot(1,2,2)
    plt.plot(rates, params, marker='s', label='Parameter Count', color='r')
    plt.xlabel('Pruning Rate')
    plt.ylabel('Number of Parameters')
    plt.legend()
    
    plt.savefig("pruning_results.png")
    print("Visualization saved as 'pruning_results.png'")

In [40]:
if __name__ == "__main__":
    H = SparsePauliOp.from_list([("ZIZZ", 1),("ZZII", 3),("IZZI", 1),("IIZZ", 1)]) # Toy hamiltonian
    circuit = QuantumCircuit(4)
    ansatz = QuantumCircuit(4)
    pruning_rates = [0.1, 0.3, 0.5, 0.7, 0.9]
    results = NaiveBuilder([1,1,1,1], ansatz, 3, circuit, H, Estimator(), pruning_rates)
    visualize_results(results)

Visualization saved as 'pruning_results.png'
